In [ ]:
%matplotlib inline
import os
import re
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV, cross_validate
from sklearn import model_selection, metrics
import matplotlib.pyplot as plt
import seaborn as sns

# models
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

# snorkel
from snorkel.labeling import labeling_function, PandasLFApplier, LFAnalysis, filter_unlabeled_dataframe
from snorkel.labeling.model import MajorityLabelVoter, LabelModel
from snorkel.analysis import get_label_buckets, metric_score
from snorkel.utils import probs_to_preds

majority_model = MajorityLabelVoter()
label_model = LabelModel(cardinality=2, verbose=True)

import helper as hlp
import importlib
importlib.reload(hlp)

# global variables
ABSTAIN = -1; CONTROL = 0; CASE = 1
SEED = 987

In [ ]:
# import data, which was saved as a tuple...
#export_data = (df_train_dev, df_valid, df_test, Y_dev, Y_valid,
#               L_train, L_dev, L_train_dev, L_valid,  
#               label_model, majority_model, L_test)

with open('./data_for_analysis.pkl', 'rb') as f:
    data = pickle.load(f)
    
df_train_dev = data[0]; df_valid = data[1]; df_test = data[2]
Y_dev = data[3]; Y_valid = data[4]
L_train = data[5]; L_dev = data[6]; L_train_dev = data[7]; L_valid = data[8]
label_model = data[9]; majority_model = data[10]
L_test = data[11]

# Prepare Model Features

## Outcome  
Although we're primarily depending on the Generative model for labels, we can still leverage our manually adjudicated information for more robust information - something is better than nothing, right? 

In [ ]:
# find observed values from label model probabilities that are closest to 0 or 1 and make 
#    manually-adjudicated labels slightly closer to 0 or 1, respectively
label_model_probs = label_model.predict_proba(L_train_dev)[:, CASE]
lower_limit = 0.95 * np.min(label_model_probs)
upper_limit = 0.95 * (1-np.max(label_model_probs)) + np.max(label_model_probs)

In [ ]:
# store on dataframe, using manual adjudication if available
df_train_dev['outcome_generative_model'] = label_model_probs
df_train_dev['outcome'] = np.where(pd.isnull(df_train_dev['label']), # if label is missing...
                                           # use generative model
                                           df_train_dev['outcome_generative_model'], 
                                           # otherwise, use manually-adjudicated label 
                                           # but with offset for regression-based models
                                           np.where(df_train_dev['label']=='case', upper_limit, lower_limit))

# create y variables
y_train_probs = np.array(df_train_dev['outcome'])
y_train_preds = np.where(df_train_dev['outcome'] >= 0.5, 1, 0)

y_valid_probs = label_model.predict_proba(L_valid)#[:, CASE] # only used as FYI
y_valid_preds = probs_to_preds(y_valid_probs) 

In [ ]:
# distribution of training set probabilities
plt.hist(y_train_probs, bins=100);

In [ ]:
# log-transformed distribution
assert np.min(y_train_probs > 0)
plt.hist(np.log(y_train_probs), bins=100);

In [ ]:
# event rate
np.mean(y_train_preds)

In [ ]:
sum(y_train_preds)

### FYI: Generative Model Performance on Validation Set

In [ ]:
eval = pd.DataFrame({'predicted': np.round(y_valid_probs[:, CASE], 2), 
                     'actual': np.where(Y_valid==0, 'Control', 'Case')})
eval = eval.sort_values(by=['predicted', 'actual'])
eval = eval.assign(counts =eval.groupby(['predicted']).cumcount())

fig = sns.scatterplot(data=eval, x="predicted", y="counts", 
                      hue=eval["actual"].tolist(), palette="colorblind", s=100)
plt.title('Generative Model Performance in Validation Set (F1=0.73, AUC=0.96)')
plt.ylabel('Visit Counts (n)')
plt.ylim([-1, 40])
plt.xlabel('Probability(OIRD)')
plt.legend(loc='upper center')
sns.set(rc={'figure.figsize': (15, 5), 'figure.dpi': 300})

In [ ]:
print(metrics.classification_report(Y_valid, y_valid_preds, digits=3))

## Predictors  

For the deterministic model, we're keeping a generalizable set of features. We could depend on the previously-developed learning functions, but one draw-back is the amount of feature engineering that's put into that. Alternatively, we can start with the raw features, e.g., age, regular expression counts, etc. It might also be unwise to use the `nalxone_admin_prob` value due to it being created with a previous Snorkel model. 

In [ ]:
# pull in original naloxone administration info & only count "received" if "epic ip admin" or "hed" present
naloxone = pd.read_csv('../sd_structured/meds/naloxone/naloxone_exposure_pre.csv', sep='\t')
naloxone.columns = naloxone.columns.str.lower()

# collapse all visit day onto a single row
SEP = ';;'
join_as_strings = lambda x: SEP.join(map(str, x))

naloxone = naloxone.groupby(['visit_occurrence_id', 'grid', 'label']) \
    ['x_frequency', 'drug_source_value', 'x_doc_type', 'x_doc_stype'] \
    .agg(join_as_strings) \
    .reset_index()

# create binary indicator of whether naloxone received based on simple rule
naloxone['binary_naloxone_admin'] = np.where((naloxone['x_doc_type'].str.contains('HED')) | 
                                            (naloxone['x_doc_type'].str.contains('EPIC IP ADMIN')),
                                            1, 0)

# attach to train/dev and validation sets
df_train_dev = df_train_dev.merge(naloxone[['visit_occurrence_id', 'binary_naloxone_admin']], 
                                  how='left', on=['visit_occurrence_id'])
df_valid = df_valid.merge(naloxone[['visit_occurrence_id', 'binary_naloxone_admin']], 
                          how='left', on=['visit_occurrence_id'])

In [ ]:
# create numeric columns from string-based columns
df_train_dev['binary_respiratory_failure_any'] = \
    np.where(df_train_dev['respiratory_failure_any'].str.contains('1'), 1, 0)
df_valid['binary_respiratory_failure_any'] = \
    np.where(df_valid['respiratory_failure_any'].str.contains('1'), 1, 0)

df_train_dev['binary_eligible_vent'] = \
    np.where(df_train_dev['eligible_vent'].str.contains('Yes'), 1, 0)
df_valid['binary_eligible_vent'] = \
    np.where(df_valid['eligible_vent'].str.contains('Yes'), 1, 0)

# coerce only categorical column into binary
df_train_dev['binary_gender_female'] = np.where(df_train_dev['gender']=='FEMALE', 1, 0)
df_valid['binary_gender_female'] = np.where(df_valid['gender']=='FEMALE', 1, 0)

# replace missing values from naloxone join with "0"
df_train_dev = df_train_dev.fillna(value={'binary_naloxone_admin': 0})
df_valid = df_valid.fillna(value={'binary_naloxone_admin': 0})

# replace NaN values with 0 for ICD conditions
df_train_dev['binary_cond_resp_failure'] = np.where(df_train_dev['cond_resp_failure']==1, 1, 0)
df_valid['binary_cond_resp_failure'] = np.where(df_valid['cond_resp_failure']==1, 1, 0)

df_train_dev['binary_cond_sepsis'] = np.where(df_train_dev['cond_sepsis']==1, 1, 0)
df_valid['binary_cond_sepsis'] = np.where(df_valid['cond_sepsis']==1, 1, 0)

df_train_dev['binary_cond_cva'] = np.where(df_train_dev['cond_cva']==1, 1, 0)
df_valid['binary_cond_cva'] = np.where(df_valid['cond_cva']==1, 1, 0)

df_train_dev['binary_cond_resp_disease'] = np.where(df_train_dev['cond_resp_disease']==1, 1, 0)
df_valid['binary_cond_resp_disease'] = np.where(df_valid['cond_resp_disease']==1, 1, 0)

df_train_dev['binary_cond_cv_disease'] = np.where(df_train_dev['cond_cv_disease']==1, 1, 0)
df_valid['binary_cond_cv_disease'] = np.where(df_valid['cond_cv_disease']==1, 1, 0)

## Create Data Matrices  

In [ ]:
# specify columns for model building
cols_binary = df_train_dev.columns[df_train_dev.columns.str.contains('binary_')]
cols_counts = df_train_dev.columns[df_train_dev.columns.str.contains('counts_')]

cols = ['age_on_admission'] #'naloxone_admin_prob'
cols.extend(cols_binary)
cols.extend(cols_counts)
cols

In [ ]:
# subset columns
X_train = df_train_dev[cols]
X_valid = df_valid[cols]

# also, some of the "counts" variables didn't have any results because those patients didn't have charts
#    consider imputing "0" here, too
X_train.fillna(0, inplace=True)
X_valid.fillna(0, inplace=True)

In [ ]:
# scale data 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_valid = sc.transform(X_valid)

# Build Deterministic Model(s)

## Classifiers: Off-the-Shelf

We begin with off-the-shelf models that don't require hyper-parameter tuning at this time. This will give us a start in determining which models might be worth hyper-parameter tuning.

In [ ]:
# prepare models
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('RF', RandomForestClassifier()))
models.append(('NB', GaussianNB()))
models.append(('NN', MLPClassifier()))

In [ ]:
# evaluate each model in turn
results = dict()
names = []
scoring = ('f1', 'roc_auc', 'neg_mean_squared_error')
y = y_train_preds # binary classification

for name, model in models:
    kfold = model_selection.KFold(n_splits=5, random_state=SEED, shuffle=True)
    # apply multiple scoring metrics
    cv_results = model_selection.cross_validate(model, X_train, y, cv=kfold, scoring=scoring, n_jobs=-1)
    # store results
    names.append(name)
    results[name] = cv_results
    for score in scoring:
        print('{:<5s}{:<25s}{:^15.3f}{:>5.3f}'.format(name, score, cv_results['test_' + score].mean(), 
                                                      cv_results['test_' + score].std()))

In [ ]:
# algorithm comparison
for score in scoring:
    results_partial = []
    for name in names:
        results_partial.append(results[name]['test_' + score])
    fig = plt.figure()
    fig.suptitle('Algorithm Comparison based on ' + score)
    ax = fig.add_subplot(111)
    plt.boxplot(results_partial)
    plt.ylabel(score)
    ax.set_xticklabels(names)
    plt.show()

Using off-the-shelf models with no hyper-parameter tuning, it appears the LDA, Random Forest, & Neural Network are the best models across F1 & AUC metrics. I've never seen LDA out-perform RF, so I'll tune RF & NN classifiers further. 

## Regressors: Off-the-Shelf

In [ ]:
models = []
models.append(('LR', LinearRegression()))
models.append(('RF', RandomForestRegressor()))
models.append(('NN', MLPRegressor()))

In [ ]:
results = []
names = []
scoring = 'neg_mean_squared_error' 
y = np.log(y_train_probs)

for name, model in models:
    kfold = model_selection.KFold(n_splits=5, random_state=SEED, shuffle=True)
    cv_results = model_selection.cross_val_score(model, X_train, y, 
                                                 cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
plt.ylabel(scoring)
ax.set_xticklabels(names)
plt.show()

Similar to the classifiers, it looks like RF & NN are the best performers, and we could try more hyper-parameter tuning. 

In [ ]:
sk_rfr = RandomForestRegressor()
sk_rfr.fit(X=X_train, y=np.log(y_train_probs))

In [ ]:
y_pred_proba = np.exp(sk_rfr.predict(X_valid)) # convert back to probability

eval = pd.DataFrame({'predicted': np.round(y_pred_proba, 2), 'actual': Y_valid})
eval = eval.sort_values(by=['predicted', 'actual'])
eval = eval.assign(counts =eval.groupby(['predicted']).cumcount())

fig = sns.scatterplot(data=eval, x="predicted", y="counts", 
                      hue=eval["actual"].tolist(), palette="colorblind", s=100)
plt.ylabel('Counts')
plt.xlabel('Predicted Value')

In [ ]:
# assess classifier-like metrics
y_pred = np.where(y_pred_proba >= 0.5, 1, 0)

print(metrics.classification_report(Y_valid, y_pred, digits=3))

In [ ]:
# attempt sample weights with by downweighting probabilities closer to 0.5
weights = np.abs(df_train_dev['outcome_generative_model']-0.5)

In [ ]:
sk_rfr = RandomForestRegressor()
sk_rfr.fit(X=X_train, y=np.log(y_train_probs), sample_weight=weights)

In [ ]:
y_pred_proba = np.exp(sk_rfr.predict(X_valid)) # convert back to probability

eval = pd.DataFrame({'predicted': np.round(y_pred_proba, 2), 'actual': Y_valid})
eval = eval.sort_values(by=['predicted', 'actual'])
eval = eval.assign(counts =eval.groupby(['predicted']).cumcount())

fig = sns.scatterplot(data=eval, x="predicted", y="counts", 
                      hue=eval["actual"].tolist(), palette="colorblind", s=100)
plt.ylabel('Counts')
plt.xlabel('Predicted Value')

In [ ]:
y_pred = np.where(y_pred_proba >= 0.5, 1, 0)
print(metrics.classification_report(Y_valid, y_pred, digits=3))

## Nested Cross-Validation

In [ ]:
if os.path.exists('./cv_results.pkl'):
    cv_results = pickle.load(open('./cv_results.pkl', 'rb'))
else:
    cv_results = dict()

In [ ]:
# global values
K_i = 3         # inner folds
K_o = 10        # outer folds
verbosity = 0   # do not print when exploring full set of values

# test values for debugging
#K_i = 3
#K_o = 3
#verbosity = 2

In [ ]:
importlib.reload(hlp)

### Random Forest Classifier - Unweighted

In [ ]:
model = RandomForestClassifier()
params = [{'n_estimators': [250],
           'class_weight': [None, 'balanced', 'balanced_subsample', {0: 0.99, 1: 0.01}, {0: 0.01, 1: 0.99}],
           'max_depth': [None, 5, 10, 50],
           'max_features': [None, 'sqrt', 'log2']}]

rfc = hlp.nested_cv(X_train, y_train_preds, model, params, 
                    k_outer=K_o, k_inner=K_i, verbosity=verbosity)

In [ ]:
# save values in directory
name = 'RandomForestClassifier-Unweighted'
cv_results[name] = {'model': model,
                   'params': params, 
                   'results': rfc}

In [ ]:
# save to disk
with open('cv_results.pkl', 'wb') as f:
    pickle.dump(cv_results, f)

In [ ]:
hlp.summarize_model_performance(cv_results['RandomForestClassifier-Unweighted']['results'])
cv_results['RandomForestClassifier-Unweighted']['results']['best_params_inner_cv']

### Random Forest Classifier - Weighted

In [ ]:
# attempt sample weights with by downweighting probabilities closer to 0.5
weights = np.abs(df_train_dev['outcome_generative_model']-0.5)

rfc_weighted = hlp.nested_cv(X_train, y_train_preds, model, params, sample_weight=weights, 
                             k_outer=K_o, k_inner=K_i, verbosity=verbosity)

In [ ]:
name = 'RandomForestClassifier-Weighted'
cv_results[name] = {'model': model,
                   'params': params, 
                   'results': rfc_weighted}

In [ ]:
with open('cv_results.pkl', 'wb') as f:
    pickle.dump(cv_results, f)

In [ ]:
hlp.summarize_model_performance(cv_results['RandomForestClassifier-Weighted']['results'])
cv_results['RandomForestClassifier-Weighted']['results']['best_params_inner_cv']

### Neural Network Classifier

In [ ]:
model = MLPClassifier(max_iter=250, solver='adam')
params = [{'hidden_layer_sizes': [(10,30,10), (10,30,30,30,10), (25,50,50,50,25), (20,)],
           'activation': ['tanh', 'relu'],
           'alpha': [0.0001, 0.05],
           'learning_rate': ['constant'],#, 'adaptive'],
           'random_state': [123, 987]}
         ]

nnc = hlp.nested_cv(X_train, y_train_preds, model, params, 
                    k_outer=K_o, k_inner=K_i, verbosity=verbosity)

In [ ]:
name = 'NeuralNetworkClassifier'
cv_results[name] = {'model': model,
                   'params': params, 
                   'results': nnc}

In [ ]:
hlp.summarize_model_performance(nnc)
nnc['best_params_inner_cv']

In [ ]:
with open('cv_results.pkl', 'wb') as f:
    pickle.dump(cv_results, f)

### Random Forest Regressor

In [ ]:
model = RandomForestRegressor()
params = [{'n_estimators': [250],
           'max_depth': [None, 5, 10, 50],
           'max_features': [None, 'sqrt', 'log2']}]

rfr = hlp.nested_cv(X_train, np.log(y_train_probs), model, params, tune_metric='neg_mean_squared_error',
                    k_outer=K_o, k_inner=K_i, verbosity=verbosity)

In [ ]:
name = 'RandomForestRegressor'
cv_results[name] = {'model': model,
                   'params': params, 
                   'results': rfr}

In [ ]:
hlp.summarize_model_performance(cv_results['RandomForestRegressor']['results'])
cv_results['RandomForestRegressor']['results']['best_params_inner_cv']

In [ ]:
with open('cv_results.pkl', 'wb') as f:
    pickle.dump(cv_results, f)

### Neural Network Regressor

In [ ]:
model = MLPRegressor(max_iter=300, solver='adam') # increased iter d/t convergence issues

params = [{'hidden_layer_sizes': [(10,30,10), (10,30,30,30,10), (20,)],
           'activation': ['tanh', 'relu'],
           'alpha': [0.0001, 0.05],
           #'learning_rate': ['constant'],#, 'adaptive'], # only used with 'sgd' solver
           'random_state': [123, 987]}
         ]

nnr = hlp.nested_cv(X_train, np.log(y_train_probs), model, params, tune_metric='neg_mean_squared_error',
                    k_outer=K_o, k_inner=K_i, verbosity=verbosity)

In [ ]:
name = 'NeuralNetworkRegressor'
cv_results[name] = {'model': model,
                   'params': params, 
                   'results': nnr}

In [ ]:
hlp.summarize_model_performance(cv_results['NeuralNetworkRegressor']['results'])
cv_results['NeuralNetworkRegressor']['results']['best_params_inner_cv']

In [ ]:
with open('cv_results.pkl', 'wb') as f:
    pickle.dump(cv_results, f)

## Nested CV Comparison

In [ ]:
for score in ['acc', 'f1', 'mse', 'auc']:
    results_partial = []
    for name in cv_results.keys():
        results_partial.append(cv_results[name]['results'][score])
    fig = plt.figure()
    fig.suptitle('Algorithm Comparison based on ' + score)
    ax = fig.add_subplot(111)
    plt.boxplot(results_partial)
    plt.ylabel(score)
    ax.set_xticklabels(cv_results.keys())
    plt.show()

In general, the classifiers did better than the regressors based on F1 & AUC. The MSE & Accuracy aren't too much better than random guessing given the low event rate. It also appears the random forests perform better than the neural networks. 

In [ ]:
# prepare final figures for thesis
cv_results = pickle.load(open('./cv_results.pkl', 'rb'))

In [ ]:
results_partial = []
for name in cv_results.keys():
    results_partial.append(cv_results[name]['results']['f1'])
fig = plt.figure(figsize=(16, 5))
fig.suptitle('Algorithm Comparison based on F1 Score')
ax = fig.add_subplot(111)
plt.boxplot(results_partial)
plt.ylabel('F1')
ax.set_xticklabels(cv_results.keys())
plt.show()

In [ ]:
results_partial = []
for name in cv_results.keys():
    results_partial.append(cv_results[name]['results']['auc'])
fig = plt.figure(figsize=(16, 5))
fig.suptitle('Algorithm Comparison based on AUROC')
ax = fig.add_subplot(111)
plt.boxplot(results_partial)
plt.ylabel('AUROC')
ax.set_xticklabels(cv_results.keys())
plt.show()

In [ ]:
results_partial = []
for name in cv_results.keys():
    results_partial.append(cv_results[name]['results']['mse'])
fig = plt.figure(figsize=(16, 5))
fig.suptitle('Algorithm Comparison based on Mean Squared Error')
ax = fig.add_subplot(111)
plt.boxplot(results_partial)
plt.ylabel('Mean Squared Error')
ax.set_xticklabels(cv_results.keys())
plt.show()

## Build "Best" Models on Full Training Data

In [ ]:
#cv_results['RandomForestClassifier-Weighted']['results']['best_params_inner_cv']
# class_weight={0: 0.99, 1: 0.01}, max_depth=50, max_features=None, 

In [ ]:
#cv_results['RandomForestClassifier-Unweighted']['results']['best_params_inner_cv']
# class_weight=None, max_depth=10, max_features=None, 

In [ ]:
#cv_results['RandomForestRegressor']['results']['best_params_inner_cv']
# max_depth=10, max_features=None, 

In [ ]:
#cv_results['NeuralNetworkClassifier']['results']['best_params_inner_cv']
# activation='tanh', alpha=0.0001, hidden_layer_sizes=(10, 30, 10), learning_rate='constant', random_state=123,

In [ ]:
#cv_results['NeuralNetworkRegressor']['results']['best_params_inner_cv']
# activation='tanh', alpha=0.05, hidden_layer_sizes=(10, 30, 30, 30, 10), random_state=987, 

In [ ]:
best_rfcw = RandomForestClassifier(n_estimators=1000, random_state=SEED, 
                                   class_weight={0: 0.99, 1: 0.01}, max_depth=50, max_features=None)

weights = np.abs(df_train_dev['outcome_generative_model']-0.5)
best_rfcw.fit(X_train, y_train_preds, sample_weight=weights)

In [ ]:
best_rfcu = RandomForestClassifier(n_estimators=1000, random_state=SEED, 
                                   class_weight=None, max_depth=10, max_features=None)
best_rfcu.fit(X_train, y_train_preds)

In [ ]:
best_rfr = RandomForestRegressor(n_estimators=1000, random_state=SEED,
                                max_depth=10, max_features=None)
best_rfr.fit(X_train, np.log(y_train_probs))

In [ ]:
best_nnc = MLPClassifier(max_iter=10000, solver='adam', 
                        activation='tanh', alpha=0.0001, hidden_layer_sizes=(10, 30, 10),
                        learning_rate='constant', random_state=123)
best_nnc.fit(X_train, y_train_preds)

In [ ]:
best_nnr = MLPRegressor(max_iter=10000, solver='adam', 
                       activation='tanh', alpha=0.05, hidden_layer_sizes=(10, 30, 30, 30, 10), random_state=987)
best_nnr.fit(X_train, np.log(y_train_probs))

In [ ]:
y_pred = best_rfcw.predict(X_train)
y_pred

In [ ]:
len(np.unique(y_pred)) > 2

## Performance of Best Models

### Training Set Performance

In [ ]:
# performance in training set (should be highly fit)
# classifiers
for model in [best_rfcw, best_rfcu, best_nnc]:
    y_pred = model.predict(X_train)
    print(model)
    print(metrics.classification_report(y_train_preds, y_pred, digits=3))
    print(metrics.roc_auc_score(y_train_preds, y_pred))

In [ ]:
# regressors
for model in [best_rfr, best_nnr]:
    y_pred = model.predict(X_train)
    y_pred = np.exp(y_pred)
    y_pred = np.where(y_pred >= 0.5, 1, 0)
    print(model)
    print(metrics.classification_report(y_train_preds, y_pred, digits=3))
    print(metrics.roc_auc_score(y_train_preds, y_pred))

### Validation Set Performance

Take all 5 "best" models & evaluate in the Validation set. The winner can be used to re-build using the combination of Training/Development/Validation sets. Then, evaluate in Test set. 

In [ ]:
# classifiers
for model in [best_rfcw, best_rfcu, best_nnc]:
    y_pred = model.predict(X_valid)
    print(model)
    print(metrics.classification_report(Y_valid, y_pred, digits=3))
    print(metrics.roc_auc_score(Y_valid, y_pred))

Best classifier = weighted random forest based on F1, Accuracy, & AUC

In [ ]:
# regressors
for model in [best_rfr, best_nnr]:
    y_pred = model.predict(X_valid)
    y_pred = np.exp(y_pred)
    y_pred = np.where(y_pred >= 0.5, 1, 0)
    print(model)
    print(metrics.classification_report(Y_valid, y_pred, digits=3))
    print(metrics.roc_auc_score(Y_valid, y_pred))

Best regressor = neural network. However, it performed really poorly in the training set. That makes me worried that perhaps it's a bit random, and it makes me want to lean toward using the random forest classifier. 

In [ ]:
# accuracy of random guess in Validation set
1-sum(Y_valid)/len(Y_valid)

In [ ]:
# probability distribution from weighted random forest classifier 
y_pred_proba = best_rfcw.predict_proba(X_valid)[:,CASE]

eval = pd.DataFrame({'predicted': np.round(y_pred_proba, 2), 
                     'actual': np.where(Y_valid==0, 'Control', 'Case')})
eval = eval.sort_values(by=['predicted', 'actual'])
eval = eval.assign(counts =eval.groupby(['predicted']).cumcount())

fig = sns.scatterplot(data=eval, x="predicted", y="counts", 
                      hue=eval["actual"].tolist(), palette="colorblind", s=100)
plt.title('Discriminative Model Performance in Validation Set (F1=0.80, AUC=0.92)')
plt.ylabel('Visit Counts (n)')
plt.ylim([-1, 40])
plt.xlabel('Probability(OIRD)')
plt.legend(loc='upper center')
sns.set(rc={'figure.figsize': (15, 5), 'figure.dpi': 300})

In [ ]:
# important features from random forest
top_n = 10
importances = best_rfcw.feature_importances_
std = np.std([tree.feature_importances_ for tree in best_rfcw.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1][:top_n]

print("Feature ranking:")
for f in range(top_n):
    print("%d. %s (%f)" % (f + 1, cols[indices[f]], importances[indices[f]]))

## Bias Correction  

Because the event rate of OIRD in the training/dev set on which the models were built is so much lower than the validation set, it might be worth considering a bias correction. However, the test set should have an event rate closer to the training/dev, so I've decided we don't actually need to do this step. 

## Manual Review of Misclassifications

In [ ]:
y_pred_proba = best_rfcw.predict_proba(X_valid)[:,CASE]

In [ ]:
review_valid = df_valid.copy()
review_valid['final_model_prob'] = y_pred_proba

# subset to those with probability >= 0.5 but labeled as control on manual review
review_valid = review_valid[(review_valid['final_model_prob'] >= 0.5) & \
                            (review_valid['label']=='control')] \
                            .sort_values('final_model_prob', ascending=False)

In [ ]:
for i in range(review_valid.shape[0]):
    print(review_valid['visit_occurrence_id'].iloc[i])
    print(review_valid['final_model_prob'].iloc[i])
    print(review_valid['review_notes'].iloc[i])

## Re-Fit "Best" Model All Data (except Test Set)

Based on the performance in the Validation Set, the Weighted Random Forest Classification model performed best. We are re-building the model with all of the data except for the Test Set.  

In [ ]:
maj_probs_train = majority_model.predict_proba(L=L_train_dev)[:, CASE]
maj_probs_valid = majority_model.predict_proba(L=L_valid)[:, CASE]

df_train_dev['majority_model_label'] = maj_probs_train
df_valid['majority_model_label'] = maj_probs_valid

In [ ]:
# the train/dev set has an "outcome_generative_model" column that is used for creating
#   weights in the weighted RF model - replicating that in the validation set before merging
df_valid_temp = df_valid.copy()
df_valid_temp['outcome_generative_model'] = y_valid_probs[:, CASE]

# merge train/dev and validation sets 
df_final = df_train_dev.append(df_valid_temp, sort=False)

In [ ]:
# update the 'outcome' column now that validation is also there
df_final['outcome'] = np.where(pd.isnull(df_final['outcome']), # if label missing...
                                  # pull from manual 'label' (same as above code)
                                  np.where(df_final['label']=='case', upper_limit, lower_limit), 
                                  # otherwise, keep it what it is
                                  df_final['outcome'])

In [ ]:
# create y variables
y_final_probs = np.array(df_final['outcome'])
y_final_preds = np.where(df_final['outcome'] >= 0.5, 1, 0)

In [ ]:
# prepare features - code taken from above

# subset columns
X_final = df_final[cols]

# also, some of the "counts" variables didn't have any results because those patients didn't have charts
#    consider imputing "0" here, too
X_final.fillna(0, inplace=True)

# scale data 
X_final = sc.transform(X_final)

In [ ]:
# store weights
weights_final = np.abs(df_final['outcome_generative_model']-0.5)

In [ ]:
model_final = RandomForestClassifier(n_estimators=1000, random_state=SEED, 
                                     class_weight={0: 0.99, 1: 0.01}, max_depth=50, max_features=None)

model_final.fit(X_final, y_final_preds, sample_weight=weights_final)

In [ ]:
# performance in training set (should be highly fit)
y_pred = model_final.predict(X_final)
y_pred_proba_final = model_final.predict_proba(X_final)[:,CASE]
print(model_final)
print(metrics.classification_report(y_final_preds, y_pred, digits=3))
print(metrics.roc_auc_score(y_final_preds, y_pred))

In [ ]:
# store predictions on data set & export for prediction model development 
df_final['snorkel_deterministic_model_prob'] = y_pred_proba_final
df_final.to_csv('./train_dev_valid_set_with_predicted_labels.csv', index=False)

# Apply Predictions from Final Deterministic Model to Test Set

In [ ]:
# repeating code from above on train/dev and valid sets
_, _, _, df_test = hlp.reattach_numeric_data(df_train_dev, df_train_dev, df_valid, df_test)

df_test = df_test.merge(naloxone[['visit_occurrence_id', 'binary_naloxone_admin']], 
                          how='left', on=['visit_occurrence_id'])
df_test['binary_respiratory_failure_any'] = \
    np.where(df_test['respiratory_failure_any'].str.contains('1'), 1, 0)
df_test['binary_eligible_vent'] = \
    np.where(df_test['eligible_vent'].str.contains('Yes'), 1, 0)
df_test['binary_gender_female'] = np.where(df_test['gender']=='FEMALE', 1, 0)
df_test = df_test.fillna(value={'binary_naloxone_admin': 0})
df_test['binary_cond_resp_failure'] = np.where(df_test['cond_resp_failure']==1, 1, 0)
df_test['binary_cond_sepsis'] = np.where(df_test['cond_sepsis']==1, 1, 0)
df_test['binary_cond_cva'] = np.where(df_test['cond_cva']==1, 1, 0)
df_test['binary_cond_resp_disease'] = np.where(df_test['cond_resp_disease']==1, 1, 0)
df_test['binary_cond_cv_disease'] = np.where(df_test['cond_cv_disease']==1, 1, 0)

X_test = df_test[cols]
X_test.fillna(0, inplace=True)
X_test = sc.transform(X_test)

In [ ]:
y_pred_binar_test = model_final.predict(X_test)
y_pred_proba_test = model_final.predict_proba(X_test)[:,CASE]

In [ ]:
sum(y_pred_binar_test)

In [ ]:
sum(y_pred_binar_test)/len(y_pred_binar_test)

In [ ]:
plt.hist(y_pred_proba_test, bins=100);

In [ ]:
non_small = y_pred_proba_test[np.where(y_pred_proba_test > 0.01)]
plt.hist(non_small, bins=100);

In [ ]:
# keep generative model probs for comparison
df_test['snorkel_generative_model_prob'] = label_model.predict_proba(L_test)[:, CASE]

# store majority model probs
df_test['majority_model_label'] = majority_model.predict_proba(L_test)[:, CASE]

In [ ]:
# export
df_test['snorkel_deterministic_model_prob'] = y_pred_proba_test
df_test.to_csv('./test_set_with_predicted_labels.csv', index=False)

# Manuscript Aids

In [ ]:
# load updated training/dev/valid data after labeling from previous round
df_train, df_dev, df_valid, df_test = hlp.load_data(round=5)

# re-attach numeric data to reflect any updated rules
df_train, df_dev, df_valid, df_test = hlp.reattach_numeric_data(df_train, df_dev, df_valid, df_test)

# keep confounding diagnoses visits available
confounding_diagnosis_present = pd.read_csv('../sd_structured/icd/visits_with_confounding_icd_codes.csv')

# made changes in code in to loading confounding diagnoses, so eliminating the redundant columns in corrected
#    validation set
df_valid.drop(['condition_start_date', 'cva', 'sepsis'], axis = 1, inplace = True)

In [ ]:
print(df_train.shape[0])
print(len(np.unique(df_train['grid'])))

In [ ]:
print(df_dev.shape[0])
print(len(np.unique(df_dev['grid'])))

In [ ]:
print(df_valid.shape[0])
print(len(np.unique(df_valid['grid'])))

In [ ]:
print(df_test.shape[0])
print(len(np.unique(df_test['grid'])))

In [ ]:
df_test.shape